In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros(img.shape, img.dtype)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    global lines
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img


In [2]:
def mask_boundary(shape):
    size_y, size_x = shape
    
    # left bottom, left top, right top, right bottom
    return np.array([[(size_x * 0.10, size_y - 40), \
                      (size_x * 0.45, size_y * 0.60), \
                      (size_x * 0.55, size_y * 0.60), \
                      (size_x * 0.97, size_y - 20)]], dtype=np.int32)

def detect_lanes_image(image_name, debug = False):
    img = mpimg.imread(image_name)
    return process_image(img, debug)

def test_single(filename, debug = False):
    lanes = detect_lanes_image(filename, debug)

    plt.figure(figsize=(10,10))
    plt.imshow(lanes)
    plt.show()

def draw_ideal(ideal_line, img):
    start = ideal_line[0,0]
    end = ideal_line[0,1]
    cv2.line(img, (start[0], start[1]), (end[0], end[1]), [255,255,0], 2)

In [3]:
def image_for_canny(image):
    target_color = [225, 225, 225]
    square = np.sqrt(np.sum(np.square(target_color))) 
    diff_image = np.sqrt(np.sum(np.square(image - target_color), axis=2))
    diff_image = 255 - (diff_image / square * 255).astype(np.uint8)
    return cv2.GaussianBlur(diff_image, (3,3), 0)

# def image_for_canny(image):
#     # https://www.compuphase.com/cmetric.htm
#     # yellow
#     target_color = [250,250,50]
#     rmean = (image[:,:,0] + target_color[0]) / 2
#     diff = image - target_color
#     r = ((512 + rmean) * diff[:,:,0] ** 2) / 256
#     g = 4 * diff[:,:,1] ** 2
#     b = ((767 - rmean) * diff[:,:,2] ** 2) / 256

#     x = np.sqrt(r + g + b)
#     x = x / np.max(x) * 255
#     #plt.imshow(x, cmap='gray')
#     return x.astype(np.uint8)

def process_image(image, debug=False):
    global raw_image, gray_blur_image, debug_image
    
    # remove any alpha channel from the image and just keep the RGB
    raw_image = image[:,:,:3]
    
    blur_image = image_for_canny(raw_image)
    edges = canny(blur_image, 70, 160)
    
    if debug:
        plt.figure(figsize=(10,10))
        plt.subplot(121)
        plt.imshow(blur_image, cmap='gray')
        plt.subplot(122)
        plt.imshow(edges)

        mpimg.imsave('test/1-bw.jpg', blur_image, cmap='gray')
        mpimg.imsave('test/2-canny.jpg', edges)

    vertices = mask_boundary(edges.shape)
    masked_edges = region_of_interest(edges, vertices)
    
    if debug:
        mpimg.imsave('test/3-masked_edges.jpg', masked_edges, cmap='gray')
    
    #lines_image = hough_lines(masked_edges, 2, np.pi / 180.0, threshold = 30, min_line_len = 20, max_line_gap = 45)
    lines = cv2.HoughLinesP(masked_edges, 2, 10 * np.pi / 180, 20, np.array([]), \
                            minLineLength=20, maxLineGap=30)
   
    if debug:
        masked_color = cv2.cvtColor(masked_edges, cv2.COLOR_GRAY2RGB)

        line_img = np.zeros(masked_color.shape, masked_color.dtype)
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(line_img, (x1,y1), (x2,y2), [255,0,0], 1)

        debug_image = cv2.addWeighted(masked_color, 0.8, line_img, 1, 1)
        mpimg.imsave('test/4-hough.jpg', line_img, cmap='gray')
        mpimg.imsave('test/5-hough-combined.jpg', debug_image)
        plt.figure()
        plt.imshow(line_img)
    else:
        line_img = np.zeros(raw_image.shape, raw_image.dtype)
        
    draw_lines(line_img, lines, debug = debug)

    lanes = weighted_img(line_img, raw_image)
    return lanes

def test_process(image_name, debug = False):
    image = mpimg.imread(image_name)
    process_image(image, debug)
    
    plt.show()

#test_process('frames/140.jpg', True)

In [4]:
def angle_from_ideal(lines, ideal_left, ideal_right):
    angle_lines = compute_angle(lines)
    angle_right = compute_angle(ideal_right)
    angle_left = compute_angle(ideal_left)
    return (angle_left - angle_lines, angle_right - angle_lines)
    
def compute_angle(lines):
    diff = lines[:,1] - lines[:,0]
    angle_rad = np.arctan2(diff[:,1], diff[:,0])
    # add pi to all values that are -ve angles, otherwise a line
    # from top to bottom would have different angle compared to bottom to top
    less_pi = angle_rad < 0
    angle_rad[less_pi] = angle_rad[less_pi] + 1 * np.pi
    return (angle_rad * 180 / np.pi)


In [6]:
import cv2
import pdb
import os
import timeit
import sys

def get_within_distance(lines, ideal):
#     a = y1 - y2
#     b = x2 - x1
#     c = (x1 * y2) - (x2 * y1)

#   http://www.intmath.com/plane-analytic-geometry/perpendicular-distance-point-line.php
#   http://geomalgorithms.com/a02-_lines.html

    # figure out how far away is the center of a line from the ideal line,
    # remove the outliers from it
    ideal_v = ideal[:,1] - ideal[:,0]
    a = -ideal_v[0,1] # y1 - y2
    b = ideal_v[0,0] # x2 - x1
    c = (ideal[0,0,0] * ideal[0,1,1]) - (ideal[0,1,0] * ideal[0,0,1]) # x1 * y2 - x2 * y1

    lines_v = lines[:,1] - lines[:,0]
    centers = lines[:,0] + 0.5 * (lines[:,1] - lines[:,0])
    centers_x = centers[:,0]
    centers_y = centers[:,1]

    t = abs(a * centers_x + b * centers_y + c)
    distance = t / np.sqrt(a ** 2 + b ** 2)
    
    #return lines[abs(distance - np.mean(distance)) <= 2 *  np.std(distance)]
    return lines[distance < 80]

def get_possible_lines(lines, thetas, remove_outliers = True):
    degrees_allowed = 10

    good_angles = (abs(thetas) < degrees_allowed)
    lane_lines = lines[good_angles]
#     print(lane_lines, thetas[good_angles])
    
    if remove_outliers:
        good_thetas = thetas[good_angles]
#         print('good thetas', good_thetas)
#         print('mean of good thetas', np.mean(good_thetas))
#         print('std of good thetas', np.std(good_thetas))
        
        #print('mean:', np.mean(good_thetas), np.std(good_thetas))
        no_outliers = abs(good_thetas - np.mean(good_thetas)) <= 2 * np.std(good_thetas)
        lane_lines = lane_lines[no_outliers]

#     print('after outlier:', lane_lines, thetas[good_angles])
        
    return lane_lines

def draw_lines(img, lines, thickness=10, draw_hough_edges = True, debug = True):
    global thetas_left, thetas_right, ideal_left, ideal_right
    global lines_from_hough, debug_image
    
    lines_from_hough = lines
    temp_img = img.copy()
    
    if draw_hough_edges:
        for line in lines:
            for x1,y1,x2,y2 in line:
                cv2.line(img, (x1,y1), (x2,y2), [255,0,0], thickness)

    # change lines array so that it contains start, end tuple rather than
    # 4 points
    lines = lines.reshape(-1, 2, 2)

    size_y, size_x, _ = img.shape
    center_x = size_x / 2
    center_y = int(size_y * 0.6)
    bottom_y = size_y - 40
    
    # left, bottm, right, top
    # the vector is from the top since the image has 0,0 on the left top
    # so -ve X would mean going towards left
    ideal_left = np.array([[[600, center_y], [260, bottom_y]]]).astype(int)
    ideal_right = np.array([[[center_x * 1.2, center_y], [size_x - 100, bottom_y]]]).astype(int)
    
    draw_ideal(ideal_left, img)
    draw_ideal(ideal_right, img)
        
    thetas_left, thetas_right = angle_from_ideal(lines, ideal_left, ideal_right)

#     for line, angle in zip(lines, abs(thetas_right)):
#         if angle < 15:
#             cv2.line(img, (line[0,0], line[0,1]), (line[1,0], line[1,1]), [0,255,0], 10)

#     

    right_lines = get_possible_lines(lines, thetas_right)
    left_lines = get_possible_lines(lines, thetas_left)
    
    if debug:
        line_img = np.zeros(img.shape, img.dtype)
        for line in right_lines:
            cv2.line(line_img, (line[0,0], line[0,1]), (line[1,0], line[1,1]), [0,255,0], 10)
                
        draw_ideal(ideal_right, line_img)
        combined = cv2.addWeighted(debug_image, 0.8, line_img, 1, 1)
        mpimg.imsave('test/6-right-angle.jpg', combined)

        line_img = np.zeros(img.shape, img.dtype)
        for line in left_lines:
            cv2.line(line_img, (line[0,0], line[0,1]), (line[1,0], line[1,1]), [0,255,0], 10)
                
        draw_ideal(ideal_left, line_img)
        combined = cv2.addWeighted(debug_image, 0.8, line_img, 1, 1)
        mpimg.imsave('test/7-left-angle.jpg', combined)
    
#     for line in left_lines:
#         cv2.line(img, (line[0,0], line[0,1]), (line[1,0], line[1,1]), [0,255,0], 10)

#     for line in right_lines:
#         cv2.line(img, (line[0,0], line[0,1]), (line[1,0], line[1,1]), [0,255,255], 10)
#         cv2.line(temp_img, (line[0,0], line[0,1]), (line[1,0], line[1,1]), [0,255,255], 10)

    #print('Finding left distance')
    left_nearby_lines = get_within_distance(left_lines, ideal_left)
    #print('Finding right distance')
    right_nearby_lines = get_within_distance(right_lines, ideal_right)

    if debug:
        line_img = np.zeros(img.shape, img.dtype)
        for line in right_nearby_lines:
            cv2.line(line_img, (line[0,0], line[0,1]), (line[1,0], line[1,1]), [0,255,0], 10)
                
        combined = cv2.addWeighted(debug_image, 0.8, line_img, 1, 1)
        mpimg.imsave('test/8-right-distance.jpg', combined)
        
        plt.figure()
        plt.title('Right (Good Distance)')
        plt.imshow(combined)

        line_img = np.zeros(img.shape, img.dtype)
        for line in left_nearby_lines:
            cv2.line(line_img, (line[0,0], line[0,1]), (line[1,0], line[1,1]), [0,255,0], 10)
                
        combined = cv2.addWeighted(debug_image, 0.8, line_img, 1, 1)
        mpimg.imsave('test/9-left-distance.jpg', combined)
        plt.figure()
        plt.title('Left (Good Distance)')
        plt.imshow(combined)
    
#     print('Left:', left_lines, 'nearby', left_nearby_lines)
#     print('Right:', right_lines, 'nearby', right_nearby_lines)

    
#     only_angle = weighted_img(raw_image, temp_img)
#     print('only angle based')
#     plt.imshow(only_angle)

    draw_lane_line(img, right_nearby_lines, center_y, size_y, 9 if debug else 0)
    draw_lane_line(img, left_nearby_lines, center_y, size_y, 10 if debug else 0)
    
#     for start, end in left_nearby_lines:
#         cv2.line(img, (start[0], start[1]), (end[0],end[1]), [255,0,255], 10)
                
#     for start, end in right_nearby_lines:
#         cv2.line(img, (start[0], start[1]), (end[0],end[1]), [0,255,255], 10)
            
    return img

def draw_lane_line(img, side_lines, center_y, size_y, debug = 0):
    angle = compute_angle(side_lines)
    length = np.sqrt(np.sum(np.square(side_lines[:,1] - side_lines[:,0]), axis=1))
    mean_angle = np.average(angle, weights=length) * np.pi / 180
    
    if mean_angle < 0:
        mean_angle = mean_angle + np.pi
        #print('mean_angle was < 0 and now it is', mean_angle)
    
    centers = side_lines[:,0] + (side_lines[:,1] - side_lines[:,0]) * 0.5
    mean_x, mean_y = np.mean(centers, axis=0).astype(int)
    
    bottom = (int(mean_x + size_y * np.cos(mean_angle)), \
                   int(mean_y + size_y * np.sin(mean_angle)))
    
    # use triangle formula to find the top x, y. Straight
    # line from the mid to the center_x is known by hypotneuse is not known
    side_len = mean_y - center_y
    hypo = side_len / np.sin(mean_angle)
    
    top = (int(mean_x - hypo * np.cos(mean_angle)), \
                   int(mean_y - hypo * np.sin(mean_angle)))

    cv2.line(img, (mean_x, mean_y), bottom, [0,0,255], 10)
    cv2.line(img, (mean_x, mean_y), top, [0,0,255], 10)
    
    if debug > 0:
        line_img = np.zeros(img.shape, img.dtype)
        cv2.line(line_img, (mean_x, mean_y), bottom, [255,0,255], 10)
        cv2.line(line_img, (mean_x, mean_y), top, [0,255,255], 10)

        combined = cv2.addWeighted(debug_image, 0.8, line_img, 1, 1)
        mpimg.imsave('test/%d.jpg' % (debug), combined)
        plt.figure()
        plt.imshow(combined)
        
#test_single('frames/140.jpg', True)

In [7]:
def detect_lanes_image(image_name):
    img = mpimg.imread(image_name)
    return process_image(img)

def show_all_files():
    files = os.listdir('test_images')
    for file in files:
        lanes = detect_lanes_image("test_images/{0}".format(file))
        plt.figure(figsize=(10,10))
        plt.imshow(lanes)
    plt.show()
    
def test_single(filename):
    lanes = detect_lanes_image(filename)

    plt.figure(figsize=(10,10))
    plt.imshow(lanes)
    plt.show()

#show_all_files()

In [9]:
from moviepy.editor import VideoFileClip
from glob import glob
import os

def mark_lanes_video(video_filename):
    clip = VideoFileClip(video_filename)
    video_with_lanes = clip.fl_image(process_image)
    
    output = os.path.splitext(video_filename)
    %time video_with_lanes.write_videofile(output[0] + "-Lanes" + output[1], audio=False)
    
def generate_frames(video_filename):
    for file in glob('frames/*.jpg'):
        os.remove(file)
        
    clip = VideoFileClip(video_filename)
    for i,image in enumerate(clip.iter_frames()):
        mpimg.imsave('frames/%d.jpg' %(i), image)
        mpimg.imsave('frames/%d-lane.jpg' % (i), process_image(image))
        print('%d generated' % (i), end='\r')
    
mark_lanes_video('challenge.mp4')
#generate_frames('challenge.mp4')
print('done')


[MoviePy] >>>> Building video challenge-Lanes.mp4
[MoviePy] Writing video challenge-Lanes.mp4


100%|████████████████████████████████████████████████████████████████████████████████| 251/251 [00:15<00:00, 16.42it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge-Lanes.mp4 

Wall time: 16 s
done


In [ ]:
def image_for_canny(image):
    target_color = [225, 225, 225]
    square = np.sqrt(np.sum(np.square(target_color))) 
    diff_image1 = np.sqrt(np.sum(np.square(image - target_color), axis=2))
    diff_image1 = 255 - (diff_image1 / square * 255).astype(np.uint8)
    
    mpimg.imsave('test/0-white.jpg', diff_image1, cmap='gray')

    target_color = [225, 225, 50]
    square = np.sqrt(np.sum(np.square(target_color))) 
    diff_image2 = np.sqrt(np.sum(np.square(image - target_color), axis=2))
    diff_image2 = 255 - (diff_image2 / square * 255).astype(np.uint8)
    mpimg.imsave('test/0-yellow.jpg', diff_image2, cmap='gray')

    diff_image = cv2.bitwise_and(diff_image1, diff_image2)
    blur_image = gaussian_blur(diff_image, 3)
    
    return blur_image



**Yellow and White seperate, just keep that color and then combined together**

Problem runs on frame 153. The yellow line on the left has a lot of shade from trees. But if you look at it from human eye it is still a yellow. The formula for yellow in the code can't even detect one small segment of it.

Future work: Maybe try out ratios of color e.g. R is 3 times G, which is 1.2 times B then its a yellow line

In [ ]:
def image_from_color(image, target_color):
    square = np.sqrt(np.sum(np.square(target_color))) 
    diff_image = np.sqrt(np.sum(np.square(image - target_color), axis=2))
    diff_image = diff_image * 255/441
    
    mask = diff_image > 50
    diff_image[mask] = 255
    diff_image[~mask] = 0

    return diff_image

def image_for_canny(image):
    yellow_color = [255,255,50]
    yellow = image_from_color(image, yellow_color).astype(np.uint8)

    white_color = [255,255,255]
    white = image_from_color(image, white_color).astype(np.uint8)
    
    lanes = cv2.bitwise_and(white, yellow)
    blur_image = cv2.GaussianBlur(lanes, (3, 3), 0)
    return blur_image